# Lambda³ Weather Analysis - Automated Pair Analysis System

This notebook analyzes meteorological data structure tensor interactions based on Lambda³ theory.

## Workflow
1. Retrieve necessary code from GitHub repository
2. Fetch Tokyo weather data from Open-Meteo API
3. Extract Lambda³ features
4. Execute automated analysis for all parameter pairs

## 1. Environment Setup and Dependencies Installation

In [ ]:
# Install required libraries
!pip install openmeteo-requests requests-cache retry-requests pymc arviz numba networkx -q

print("✓ Library installation completed")

## 2. Retrieve Lambda³ Code from GitHub

In [ ]:
# Clone GitHub repository
!git clone https://github.com/miosync-masa/bayesian-event-detector.git

# Set working directory
import os
os.chdir('bayesian-event-detector/sample')

# Check for available files
import glob
files = glob.glob('*.py')
print(f"Available Python files: {files}")

# Handle case where WeatherAnalysis.py doesn't exist
if 'WeatherAnalysis.py' not in files:
    print("⚠️ WeatherAnalysis_tokyo.py not found. Please upload it.")
else:
    print("✓ WeatherAnalysis.py confirmed")

## 3. Create Automated Pair Analysis Module

In [ ]:
# Create lambda3_auto_pair.py
auto_pair_code = '''# ===============================
# Lambda³ Automatic Pair Analysis System
# ===============================
from itertools import combinations
from typing import Dict, List, Tuple, Optional, Union
import pandas as pd
import numpy as np
from dataclasses import dataclass, field
from pathlib import Path

@dataclass
class PairAnalysisConfig:
    """Configuration for automatic pair analysis."""
    # Analysis parameters
    analyze_all_pairs: bool = True
    max_pairs: Optional[int] = None  # None means analyze all
    min_series_length: int = 100
    
    # Pair filtering criteria
    min_correlation: Optional[float] = None  # Filter pairs by minimum correlation
    exclude_patterns: List[str] = field(default_factory=list)  # Patterns to exclude
    include_only_patterns: List[str] = field(default_factory=list)  # If set, only these
    
    # Analysis depth
    detailed_analysis_limit: int = 5  # Number of pairs for detailed plots
    summary_only_after: int = 10  # Switch to summary mode after this many pairs
    
    # Output configuration
    save_results: bool = True
    output_dir: str = "lambda3_results"
    generate_report: bool = True
'''

# Save to file
with open('lambda3_auto_pair.py', 'w') as f:
    f.write(auto_pair_code)

print("✓ lambda3_auto_pair.py created successfully")

## 4. Fetch Tokyo Weather Data from Open-Meteo API

In [ ]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

# Setup API client
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# API call parameters
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 35.6812,
    "longitude": 139.7671,
    "hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", 
               "precipitation", "wind_speed_10m", "surface_pressure"],
    "timezone": "Asia/Tokyo",
    "start_date": "2025-06-20",
    "end_date": "2025-06-27"
}

print("🌐 Fetching data from Open-Meteo API...")
responses = openmeteo.weather_api(url, params=params)

# Process response
response = responses[0]
print(f"\n📍 Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"🏔️ Elevation: {response.Elevation()} m")
print(f"🕐 Timezone: {response.Timezone()}{response.TimezoneAbbreviation()}")

# Process time series data
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(4).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(5).ValuesAsNumpy()

# Create DataFrame
hourly_data = {
    "date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    ),
    "temperature_2m": hourly_temperature_2m,
    "relative_humidity_2m": hourly_relative_humidity_2m,
    "dew_point_2m": hourly_dew_point_2m,
    "precipitation": hourly_precipitation,
    "wind_speed_10m": hourly_wind_speed_10m,
    "surface_pressure": hourly_surface_pressure
}

hourly_dataframe = pd.DataFrame(data = hourly_data)

# Display data summary
print("\n📊 Data overview:")
print(hourly_dataframe.info())
print("\n🔍 First 5 rows:")
print(hourly_dataframe.head())

# Save to CSV
hourly_dataframe.to_csv("tokyo_weather_days.csv", index=False)
print("\n✅ CSV output completed: tokyo_weather_days.csv")

## 5. Extract Lambda³ Features

In [ ]:
# Import numpy first
import numpy as np

# Import WeatherAnalysis.py
try:
    from WeatherAnalysis import (
        L3Config, 
        calc_lambda3_features_v2,
        load_csv_data,
        validate_series_lengths
    )
    print("✓ WeatherAnalysis module imported successfully")
except ImportError as e:
    print(f"⚠️ Import error: {e}")
    print("Please ensure WeatherAnalysis.py is in the current directory")
    
# Load data
print("📂 Loading data from CSV file...")
series_dict = load_csv_data(
    "tokyo_weather_days.csv",
    time_column="date",
    value_columns=["temperature_2m", "relative_humidity_2m", "dew_point_2m", 
                   "precipitation", "wind_speed_10m", "surface_pressure"]
)

# Validate data
series_dict = validate_series_lengths(series_dict)

# Lambda³ configuration
config = L3Config(
    T=len(next(iter(series_dict.values()))),
    draws=4000,  # Reduced for Colab
    tune=4000,
    delta_percentile=95.0
)

print(f"\n⚙️ Lambda³ Configuration:")
print(f"  - Time series length: {config.T}")
print(f"  - MCMC draws: {config.draws}")
print(f"  - Jump detection threshold: {config.delta_percentile}th percentile")

# Extract features for each series
print("\n🔬 Extracting Lambda³ features...")
features_dict = {}

for name, data in series_dict.items():
    print(f"\n  Processing: {name}")
    feats = calc_lambda3_features_v2(data, config)
    
    features_dict[name] = {
        'data': data,
        'delta_LambdaC_pos': feats[0],
        'delta_LambdaC_neg': feats[1],
        'rho_T': feats[2],
        'time_trend': feats[3],
        'local_jump': feats[4]
    }
    
    # Statistics
    n_pos = np.sum(feats[0])
    n_neg = np.sum(feats[1])
    mean_tension = np.mean(feats[2])
    
    print(f"    - Positive structural changes (∆ΛC+): {n_pos}")
    print(f"    - Negative structural changes (∆ΛC-): {n_neg}")
    print(f"    - Mean tension scalar (ρT): {mean_tension:.3f}")

print("\n✅ Feature extraction completed")

## 6. Execute Automated Pair Analysis

In [ ]:
# Simple automated pair analysis implementation
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns

# Generate all possible pairs
series_names = list(series_dict.keys())
all_pairs = list(combinations(series_names, 2))

print(f"🔗 Number of pairs to analyze: {len(all_pairs)}")
print("\nPair list:")
for i, (a, b) in enumerate(all_pairs):
    print(f"  {i+1}. {a} ↔ {b}")

# Calculate synchronization rates (simplified version)
from WeatherAnalysis import calculate_sync_profile

sync_results = {}
print("\n📊 Executing synchronization rate analysis...")

for pair in all_pairs:
    a, b = pair
    
    # Calculate sync rate for positive jump events
    sync_profile, max_sync, optimal_lag = calculate_sync_profile(
        features_dict[a]['delta_LambdaC_pos'].astype(np.float64),
        features_dict[b]['delta_LambdaC_pos'].astype(np.float64),
        lag_window=10
    )
    
    sync_results[pair] = {
        'sync_rate': max_sync,
        'optimal_lag': optimal_lag,
        'profile': sync_profile
    }
    
    print(f"  {a} ↔ {b}: σₛ = {max_sync:.3f} (optimal lag: {optimal_lag})")

# Create synchronization rate matrix
n = len(series_names)
sync_matrix = np.zeros((n, n))

for i, a in enumerate(series_names):
    for j, b in enumerate(series_names):
        if i == j:
            sync_matrix[i, j] = 1.0
        elif (a, b) in sync_results:
            sync_matrix[i, j] = sync_results[(a, b)]['sync_rate']
        elif (b, a) in sync_results:
            sync_matrix[i, j] = sync_results[(b, a)]['sync_rate']

# Display heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(sync_matrix, 
            annot=True, 
            fmt='.3f',
            xticklabels=series_names,
            yticklabels=series_names,
            cmap='Blues',
            vmin=0, vmax=1,
            square=True,
            cbar_kws={'label': 'Synchronization Rate σₛ'})

plt.title('Structural Synchronization Matrix Between Weather Parameters', fontsize=16)
plt.tight_layout()
plt.show()

# Display top synchronized pairs
sorted_pairs = sorted(sync_results.items(), key=lambda x: x[1]['sync_rate'], reverse=True)

print("\n🏆 Top 5 synchronized pairs:")
for i, ((a, b), data) in enumerate(sorted_pairs[:5]):
    print(f"  {i+1}. {a} ↔ {b}: σₛ = {data['sync_rate']:.3f} (lag: {data['optimal_lag']})")

## 7. Save Results and Generate Report

In [ ]:
# Save results to CSV
import pandas as pd
from datetime import datetime

# Create DataFrame from synchronization results
sync_df = pd.DataFrame([
    {
        'series_a': pair[0],
        'series_b': pair[1],
        'sync_rate': data['sync_rate'],
        'optimal_lag': data['optimal_lag']
    }
    for pair, data in sync_results.items()
])

# Sort and save
sync_df = sync_df.sort_values('sync_rate', ascending=False)
sync_df.to_csv('lambda3_sync_results.csv', index=False)
print("✅ Synchronization results saved: lambda3_sync_results.csv")

# Generate markdown report
report = f"""# Lambda³ Weather Analysis Report
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Analysis Overview
- Analysis period: 2025-06-20 to 2025-06-27
- Location: Tokyo (35.6812°N, 139.7671°E)
- Number of parameters analyzed: {len(series_names)}
- Number of pairs analyzed: {len(all_pairs)}

## Lambda³ Theory Interpretation
Meteorological phenomena are understood not as temporal causalities but as 
interactions in the semantic space of structure tensors (Λ). High synchronization 
rates indicate structural resonance.

## Top Synchronized Pairs
| Rank | Parameter A | Parameter B | Sync Rate σₛ | Optimal Lag |
|------|-------------|-------------|--------------|-------------|
"""

for i, row in sync_df.head(10).iterrows():
    report += f"| {i+1} | {row['series_a']} | {row['series_b']} | {row['sync_rate']:.3f} | {row['optimal_lag']} |\n"

report += f"""\n## Structural Insights
The highest synchronized pair ({sync_df.iloc[0]['series_a']} ↔ {sync_df.iloc[0]['series_b']}) 
shows strong structural resonance with σₛ = {sync_df.iloc[0]['sync_rate']:.3f}.
This suggests both parameters share ∆ΛC pulsations in a common semantic space.
"""

# Save report
with open('lambda3_analysis_report.md', 'w', encoding='utf-8') as f:
    f.write(report)

print("\n✅ Analysis report saved: lambda3_analysis_report.md")
print("\n🎉 Lambda³ automated pair analysis completed!")

## 8. Download Results (For Colab)

In [ ]:
# Download result files if running in Google Colab
try:
    from google.colab import files
    
    print("📥 Making result files available for download...")
    
    # Files to download
    files_to_download = [
        'tokyo_weather_days.csv',
        'lambda3_sync_results.csv',
        'lambda3_analysis_report.md'
    ]
    
    for file in files_to_download:
        try:
            files.download(file)
            print(f"  ✓ {file}")
        except:
            print(f"  ✗ {file} - File not found")
            
except ImportError:
    print("ℹ️ Running in local environment. Files are saved in current directory.")

---

## 📚 Additional Information

### About Lambda³ Theory
- All phenomena are understood as interactions of structure tensors (Λ)
- No assumption of temporal causality; analyzes structural resonance in semantic space
- ∆ΛC represents structural changes (jumps)
- ρT (tension scalar) indicates local instability

### Customization Options
1. Change analysis period: Adjust `start_date` and `end_date`
2. Change location: Adjust `latitude` and `longitude`
3. Add parameters: Add to Open-Meteo API `hourly` list
4. Adjust analysis depth: Modify `draws` and `tune` in `L3Config`

### Troubleshooting
- Memory issues: Reduce `draws` and `tune` (e.g., to 2000)
- Long execution time: Set `max_pairs` to limit number of analyzed pairs
- Import errors: Reinstall required libraries